In [2]:
#!pip install --user --upgrade python-swiftclient

In [3]:
!find Case*/*.mat -type f -size +300M -size -600M

Case105/Case105_seg4.mat
Case108/Case108_seg10.mat
Case108/Case108_seg1.mat
Case108/Case108_seg6.mat
Case108/Case108_seg7.mat
Case10/Case10_seg8.mat
Case110/Case110_seg3.mat
Case111/Case111_seg7.mat
Case112/Case112_seg7.mat
Case113/Case113_seg7.mat
Case114/Case114_seg1.mat
Case115/Case115_seg4.mat
Case117/Case117_seg1.mat
Case119/Case119_seg4.mat
Case119/Case119_seg5.mat
Case11/Case11_seg2.mat
Case11/Case11_seg3.mat
Case121/Case121_seg3.mat
Case122/Case122_seg4.mat
Case125/Case125_seg2.mat
Case126/Case126_seg3.mat
Case127/Case127_seg2.mat
Case129/Case129_seg1.mat
Case12/Case12_seg1.mat
Case130/Case130_seg3.mat
Case13/Case13_seg2.mat
Case13/Case13_seg5.mat
Case13/Case13_seg7.mat
Case15/Case15_seg4.mat
Case28/Case28_seg8.mat
Case31/Case31_seg4.mat
Case32/Case32_seg3.mat
Case33/Case33_seg5.mat
Case35/Case35_seg4.mat
Case3/Case3_seg6.mat
Case3/Case3_seg9.mat
Case41/Case41_seg7.mat
Case43/Case43_seg6.mat
Case45/Case45_seg7.mat
Case48/Case48_seg5.mat
Case49/Case49_seg9.mat
Case52/Case52_seg7

In [4]:
!ls MGH-100P | wc -l
!find Case*/*.mat -type f -size +300M -size -600M | wc -l

117
55


In [5]:
#!pip install line_profiler
#!rm full_pipeline.py
#!rm get_features.py
!ls *.py

compute_spectrogram_sunhaoqi.py  full_pipeline.py  mtspec.py
fcn_shannon_entro.py		 get_features.py   runtime.py


In [6]:
%time teraFile = spark.read.text("TeraGen-1TB/")
%time teraFile.groupBy("value").count().take(100)
print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()
import datetime
#teraFile.groupBy("value").count().take(100)
print(datetime.datetime.now())

CPU times: user 3.18 ms, sys: 768 µs, total: 3.94 ms
Wall time: 7.86 s
CPU times: user 2.95 s, sys: 601 ms, total: 3.55 s
Wall time: 18min 55s
num exec 57
2017-07-21 11:30:08.543619


In [7]:
print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()

sc.addPyFile("fcn_shannon_entro.py")
sc.addPyFile("full_pipeline.py")
sc.addPyFile("compute_spectrogram_sunhaoqi.py")
sc.addPyFile("get_features.py")
sc.addPyFile("runtime.py")
sc.addPyFile("mtspec.py")

num exec 57


In [8]:
# Same bench on full script
import itertools
import os
import numpy as np
import hdf5storage
import spectrum
from joblib import Parallel
from joblib import delayed
from numpy.core.defchararray import lower
from numpy import floor
import swiftclient
import pandas as pd
#from compute_spectrogram_sunhaoqi import mtspecgram_shq
from get_features import get_features
from scipy.signal import butter, filtfilt, lfilter
from runtime import cellarray, matlabarray, cat, cell, arange, size, disp, find, dot, copy, length, Dict2Obj,catstr
import scipy
import multiprocessing as mp

from mtspec import mt_spectrogram

#@contextlib.contextmanager
class Timer(object):
    def __init__(self, verbose=False):
        self.verbose = verbose

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start
        self.msecs = self.secs * 1000  # millisecs
        if self.verbose:
            print('elapsed time: %f ms' % self.msecs)

import datetime
def testFunc(elt):
        startdt = datetime.datetime.now()
        try:
            #d=dict(sc._conf.getAll())
            #print(d['spark.executor.id'])

            absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
            elt=absdir+elt
            datasize=os.path.getsize(elt)
            matfile = hdf5storage.loadmat(elt)


            Fs = cellarray(int(np.round(matfile['Fs'])))
            start_time = cellarray(matfile['start_time'])
            channels = cellarray(matfile['channels'])
            data = cellarray(matfile['data'])
            
            cpu_count=mp.cpu_count()
            Fs = 200

            def strrep_s(a, x, y):
                return a.replace(x, y)

            strrep = np.vectorize(strrep_s)

            print('channels', channels)
            print(size(channels, 1))
            labels = copy(channels)

            for i in arange(1, size(channels, 1)).reshape(-1):
                x = strrep(channels[i, :], ' ', '')
                labels[i] = lower(x)

            disp('load 0')
            disp('compute 1','calculate locational differential voltage')
            movingwin = matlabarray(cat(2, 2))

            # dictobj
            params = Dict2Obj({'default': 1})

            params.pad = copy(0)
            params.fpass = copy(cat(0.5, 20))
            params.err = copy(0)
            params.trialave = copy(0)
            params.tapers = copy(cat(2, 3))
            params.Fs = copy(200)

            # spect=cell(4,4)
            # COI=numpy.empty((4, ), dtype=object)
            COI = cell(4, )
            COI[1] = cellarray([['fp1', 'f7'], ['f7', 't3'], ['t3', 't5'], ['t5', 'o1']])
            COI[2] = cellarray([['fp1', 'f3'], ['f3', 'c3'], ['c3', 'p3'], ['p3', 'o1']])
            COI[3] = cellarray([['fp2', 'f4'], ['f4', 'c4'], ['c4', 'p4'], ['p4', 'o2']])
            COI[4] = cellarray([['fp2', 'f8'], ['f8', 't4'], ['t4', 't6'], ['t6', 'o2']])

            # test
            def strcmp_s(x, y):
                return x == y

            strcmp = np.vectorize(strcmp_s)

            dataB = cell(4, 4)

            # print(data.shape,data.__class__,dataB.shape,dataB.__class__,data.dtype,dataB.dtype)
            for kk in arange(1, 4).reshape(-1):
                # print('kk',kk)
                coi = COI[kk]
                print('coi', coi, size(coi, 1), size(coi, 2), size(coi))

                for k in arange(1, size(coi, 1)).reshape(-1):
                    c1 = find(strcmp(labels, coi[k, 1]))
                    c2 = find(strcmp(labels, coi[k, 2]))
                    c1 = c1.item()
                    c2 = c2.item()

                    dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())    


            # Step 2 - spectrogram
            # Par version
            import multiprocessing as mp
            cpu_count=mp.cpu_count()
            # print(cpu_count)
            n_jobs=int(np.floor(cpu_count / 2))
            n_jobs = n_jobs  # number of cpus for parallel computing, -1 is all cpus
            verbose = True  # verbosity in parallel computing
            res = Parallel(n_jobs=n_jobs, verbose=verbose)(
                delayed(mt_spectrogram)(dataB[kk,k], Fs) for kk,k in itertools.product([1,2,3,4],[
                    1,2,3,4])
            )

            spect = cell(4, 4)

            for i, rr in enumerate(res):
                ii=i % 4
                iii=floor(i/4)
                spect[iii+1, ii + 1] = rr[0].T

            stimes=res[0][1]
            sfreqs = res[0][2]
            
            msg=spect.shape
        except Exception as e:
            print('error')
            start_time=''
            data=np.array((0,0))
            datasize=0
            
            cpu_count=0
            msg=str(e)

        enddt = datetime.datetime.now()
        dur=enddt-startdt
        return (elt,start_time,startdt,enddt,datasize,data.shape,cpu_count,dur,msg)

# Benching 8
runt=True
if (runt):
    #lfiles=!ls Case10*/*.mat
    #lfiles=!find Case*/*.mat -size -100000
    # batch with sizes
    lfiles=!find Case*/*.mat -size +800M -size -1500M
    
    print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
    n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()
    
    lsub=lfiles[:n_exec]
    
    print('num data',len(lsub))
    
    distData = sc.parallelize(lsub)
    dtd=distData.repartition(n_exec)
    distData.cache()
    dtd.cache()
    
    print('num part',dtd.getNumPartitions())
    
    #testdtd=dtd.map(lambda e: testFunc(e))
    #testdtd.cache()
    #%time lres=testdtd.take(len(lsub))    
    #print(dtd.collect(),lres,len(lres)) 
    

num exec 57
num data 57
num part 57


In [9]:
print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()
n_src=dtd.collect()
print(n_src,len(n_src))

num exec 57
['Case107/Case107_seg5.mat', 'Case107/Case107_seg3.mat', 'Case108/Case108_seg13.mat', 'Case108/Case108_seg8.mat', 'Case107/Case107_seg9.mat', 'Case109/Case109_seg4.mat', 'Case108/Case108_seg11.mat', 'Case109/Case109_seg5.mat', 'Case108/Case108_seg9.mat', 'Case101/Case101_seg5.mat', 'Case109/Case109_seg2.mat', 'Case101/Case101_seg6.mat', 'Case101/Case101_seg2.mat', 'Case101/Case101_seg4.mat', 'Case102/Case102_seg3.mat', 'Case102/Case102_seg4.mat', 'Case102/Case102_seg2.mat', 'Case102/Case102_seg1.mat', 'Case103/Case103_seg8.mat', 'Case104/Case104_seg2.mat', 'Case103/Case103_seg1.mat', 'Case103/Case103_seg4.mat', 'Case105/Case105_seg1.mat', 'Case105/Case105_seg2.mat', 'Case104/Case104_seg3.mat', 'Case113/Case113_seg10.mat', 'Case104/Case104_seg4.mat', 'Case112/Case112_seg8.mat', 'Case113/Case113_seg11.mat', 'Case113/Case113_seg1.mat', 'Case112/Case112_seg9.mat', 'Case113/Case113_seg3.mat', 'Case113/Case113_seg5.mat', 'Case113/Case113_seg12.mat', 'Case113/Case113_seg13.mat', '

In [10]:
# Same bench on full script
import itertools
import os
import numpy as np
import hdf5storage
import spectrum
from joblib import Parallel
from joblib import delayed
from numpy.core.defchararray import lower
from numpy import floor
import swiftclient
import pandas as pd
#from compute_spectrogram_sunhaoqi import mtspecgram_shq
from get_features import get_features
from scipy.signal import butter, filtfilt, lfilter
from runtime import cellarray, matlabarray, cat, cell, arange, size, disp, find, dot, copy, length, Dict2Obj,catstr
import scipy
import multiprocessing as mp

from mtspec import mt_spectrogram

def mockFunc(elt):
    import multiprocessing as mp
    startdt = datetime.datetime.now()
    #meas_t=0
    print('starting',startdt)
    #try:
    absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
    elt=absdir+elt
    print('process',elt)
    dataset=os.path.getsize(elt)
    matfile = hdf5storage.loadmat(elt)

    cpu_count=mp.cpu_count()
    Fs = cellarray(int(np.round(matfile['Fs'])))
    start_time = cellarray(matfile['start_time'])
    channels = cellarray(matfile['channels'])
    data = cellarray(matfile['data'])

    # New: Resample to 200 Hz!!! %
    Fs = Fs.item()
    P = 200
    Q = int(np.ceil(Fs))
    print('source freq', Q,'to',P)
    if Q != P:
        print('resampling')

        data=scipy.signal.resample(data.T, int(len(data.T) * P / Q))
        data = cellarray(data.T)

    else:
        print('skip resampling')

    Fs = 200

    def strrep_s(a, x, y):
        return a.replace(x, y)

    strrep = np.vectorize(strrep_s)

    print('channels', channels)
    print(size(channels, 1))
    labels = copy(channels)

    for i in arange(1, size(channels, 1)).reshape(-1):
        x = strrep(channels[i, :], ' ', '')
        labels[i] = lower(x)

    disp('load 0')
    disp('compute 1','calculate locational differential voltage')
    movingwin = matlabarray(cat(2, 2))

    # dictobj
    params = Dict2Obj({'default': 1})

    params.pad = copy(0)
    params.fpass = copy(cat(0.5, 20))
    params.err = copy(0)
    params.trialave = copy(0)
    params.tapers = copy(cat(2, 3))
    params.Fs = copy(200)

    # spect=cell(4,4)
    # COI=numpy.empty((4, ), dtype=object)
    COI = cell(4, )
    COI[1] = cellarray([['fp1', 'f7'], ['f7', 't3'], ['t3', 't5'], ['t5', 'o1']])
    COI[2] = cellarray([['fp1', 'f3'], ['f3', 'c3'], ['c3', 'p3'], ['p3', 'o1']])
    COI[3] = cellarray([['fp2', 'f4'], ['f4', 'c4'], ['c4', 'p4'], ['p4', 'o2']])
    COI[4] = cellarray([['fp2', 'f8'], ['f8', 't4'], ['t4', 't6'], ['t6', 'o2']])

    # test
    def strcmp_s(x, y):
        return x == y

    strcmp = np.vectorize(strcmp_s)

    dataB = cell(4, 4)
    for kk in arange(1, 4).reshape(-1):
        # print('kk',kk)
        coi = COI[kk]
        print('coi', coi, size(coi, 1), size(coi, 2), size(coi))

        for k in arange(1, size(coi, 1)).reshape(-1):
            c1 = find(strcmp(labels, coi[k, 1]))
            c2 = find(strcmp(labels, coi[k, 2]))
            c1 = c1.item()
            c2 = c2.item()

            dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())    


    # Step 2 - spectrogram
    import multiprocessing as mp
    n_jobs =int(np.floor(cpu_count*0.7))
    #n_jobs = -1  # number of cpus for parallel computing, -1 is all cpus

    verbose = True  # verbosity in parallel computing
    res = Parallel(n_jobs=n_jobs, verbose=verbose)(
        delayed(mt_spectrogram)(dataB[kk,k], Fs) for kk,k in itertools.product([1,2,3,4],[
            1,2,3,4])
    )

    spect = cell(4, 4)
    #print(len(res))

    for i, rr in enumerate(res):
        ii=i % 4
        iii=int(floor(i/4))
        print(iii + 1, ii + 1)
        spect[iii+1, ii + 1] = rr[0].T

    stimes = res[0][1]
    sfreqs = res[0][2]

    # Step 3 - average
    R = cell(4, 1)
    for kk in arange(1, 4).reshape(-1):
        # print('kk',kk,'size',size(spect[kk,1-1]),spect[kk,1-1].shape,tuple(size(spect[kk,1-1])))
        T = np.zeros(size(spect[kk, 1]))

        print('T', T.shape)
        for k in arange(1, 4).reshape(-1):
            # print('kk',kk,'k',k)

            T = T + spect[kk, k]
        R[kk] = T / 4



    epoch_length = dot(Fs, 2)
    window_length = dot(cat(14, 10, 6, 2), Fs)

    # Step 4 -filtering of data
    fnyq = Fs / 2  # nyquist frequency
    fc_high = 50
    fc_low = 0.5
    #  cutoff frequencies
    b, a = butter(6, cat(fc_low / fnyq, fc_high / fnyq), 'bandpass')

    for kk in arange(1, 4).reshape(-1):
        for k in arange(1, 4).reshape(-1):
            # print('kk',kk,'k',k)
            temp = dataB[kk, k]

            dataB[kk, k] = filtfilt(b, a, temp, padtype='odd', padlen=3 * (np.amax((len(a), len(b))) - 1))


    # Step 5 - features extraction
    spectwindow_length = floor(window_length / Fs / 2)

    N = length(dataB[1, 1])
    nr_epochs = floor((N - (window_length[1] - epoch_length) - Fs) / epoch_length)
    nr_features = 144

    features = cell(4, nr_features, nr_epochs)
    for kk in arange(1, 4).reshape(-1):

        spect = R[kk, 1]
        data = dataB[kk, :]

        #Par version
        #n_jobs = -1 # number of cpus for parallel computing, -1 is all cpus
        verbose = True  # verbosity in parallel computing
        res = Parallel(n_jobs=n_jobs, verbose=verbose)(
            # delayed(compute_spec_each_seg)(eeg[window_start[wi]:window_start[wi] + window_length, :], NW, Fs) for wi in range(window_num)
            delayed(get_features)(data, int(i.item()), window_length, spect, sfreqs, spectwindow_length, Fs) for i
            in arange(1, nr_epochs).reshape(-1)
        )

        print(len(res))
        for i, rr in enumerate(res):
            features[kk, :, i + 1] = rr[:]

    features = features.reshape(cat(dot(4, nr_features), length(features)))
    features = features.T            


    # Step 6
    fileName0=elt
    saveFeats = features.T

    # turn this into a pandas df
    saveFeats = pd.DataFrame(saveFeats)

    # add time stamp
    saveFeats.reset_index(inplace=True)
    fileNum = fileName0[:len(fileName0) - 4]
    import re
    fileNum = re.search('(\d+)$', fileNum).group(0)
    sampleRate = round(Fs)

    def timeStamp(ind, fileNum, sampleRate):
        start = (int(fileNum) - 1) * 99999
        row = start + int(ind)
        time = row * int(sampleRate)
        return time

    saveFeats['time'] = saveFeats['index'].apply(lambda e: timeStamp(e, fileNum, sampleRate))

    # add case id
    caseNum = int(re.search(r'\d+', fileName0).group())
    saveFeats['case'] = caseNum

    try:
        print(fileName0)
        onlyFiName = os.path.basename(fileName0)
        fp = os.path.basename(fileName0) + '.csv'
        print(fp)
        savep = absdir + 'MGH-100P/new_features_' + fp
        print('save to', savep)
        if not os.path.isfile(savep):
            saveFeats.to_csv(savep)

    except:
        print('[ERROR]', 'saving to csv')



    ## get credential
    #objStorCred = {
    #  "auth_url": "https://identity.open.softlayer.com",
    #  "project": "object_storage_20ff227d_d66c_495a_a316_cd99e80a9e6f",
    #  "projectId": "a9fb4d478e3d40a8bbd54c5a2ecf25a3",
    #  "region": "dallas",
    #  "userId": "6a4cc8251c1940179a6cccc9098a15e0",
    #  "username": "admin_fd35793e7cf915d9a5a9c768b068029cf6d720b9",
    #  "password": "kDTcKA2H(3eo5.G0",
    #  "domainId": "a350f0fe7fb44571b29305706a12c95a",
    #  "domainName": "1334933",
    #  "role": "admin"
    #}
    #conn = swiftclient.Connection(auth_version='3',key=objStorCred['password'],
    #                          authurl=objStorCred['auth_url']+'/v3',
    #                         os_options={"project_id":objStorCred['projectId'],
    #                                   "user_id":objStorCred['userId'],
    #                                   "region_name":objStorCred['region']})            
    #    
    ## copy the local csv to obj storage
    #try:
    #    container = 'MGH'
    #    print('transfer to',)
    #    with open(savep, 'r') as local:
    #        # conn.put_object(container, "Case"+str(caseNum)+"/"+onlyFiName+".csv",contents=local,content_type='text/plain')
    #        conn.put_object(container, onlyFiName + ".csv", contents=local, content_type='text/plain')
    #except:
    #    print('[ERROR]', 'saveint to object store')        

    #Parallelize spectrogram
    #n_jobs=-1
    #verbose=10
    #res = Parallel(n_jobs=n_jobs, verbose=verbose)(
    #   delayed(mt_spectrogram)(cellarray(matfile['data'][i,:]), Fs) for i in range(1,16))
    msg='No issues'
    #except Exception as e:
    #    print('error')
    #    #msg=str(e)
    #    #print('error')
    #    Fs=''
    #    start_time=''
    #    dataset=(0,0)
    #    dataB=(0,0)
    #    msg=str(e) 
    #    datasize=0
    #    meas_t=0
    #    #dataset=dataset.shape
        
        
        ## Parallelize filtering
        #out = Parallel(n_jobs=n_jobs, verbose=verbose)(
        #   delayed(filtering_data)(data[i],Fs) for i in range(16))

    enddt = datetime.datetime.now()
    dur = enddt-startdt
    return (elt,start_time,startdt,enddt,datasize,data.shape,cpu_count,dur,msg)

runt=True
if (runt):
    #!ls MGH-100P
    
    print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
    n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()
    
    #lsub=['Case1/Case1_seg12.mat','Case109/Case109_seg1.mat','Case89/Case89_seg3.mat']
    #lsub=['Case17/Case17_seg7.mat','Case17/Case17_seg3.mat','Case11/Case11_seg4.mat','Case11/Case11_seg4.mat']
   # 
    #lsub=['Case11/Case11_seg4.mat','Case11/Case11_seg4.mat','Case11/Case11_seg4.mat','Case11/Case11_seg4.mat']
    
    lfiles=!find Case*/*.mat -type f -size +900M -size -1500M
    
    
    print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
    n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()
    
    lsub=lfiles[:n_exec]
    
    print('num data',len(lsub))
    
    distData = sc.parallelize(lsub)
    dtd=distData.repartition(n_exec)
    distData.cache()
    dtd.cache()
    
    print('num part',dtd.getNumPartitions())
    
    #testdtd=dtd.map(lambda e: mockFunc(e))
    #testdtd.cache()
    #%time lres=testdtd.collect()
    
    print('source files')
    print(dtd.collect())

num exec 57
num exec 57
num data 57
num part 57
source files
['Case109/Case109_seg5.mat', 'Case10/Case10_seg10.mat', 'Case10/Case10_seg9.mat', 'Case110/Case110_seg2.mat', 'Case10/Case10_seg4.mat', 'Case10/Case10_seg6.mat', 'Case111/Case111_seg4.mat', 'Case110/Case110_seg4.mat', 'Case111/Case111_seg6.mat', 'Case111/Case111_seg3.mat', 'Case102/Case102_seg2.mat', 'Case102/Case102_seg3.mat', 'Case101/Case101_seg4.mat', 'Case101/Case101_seg5.mat', 'Case104/Case104_seg4.mat', 'Case104/Case104_seg3.mat', 'Case102/Case102_seg4.mat', 'Case104/Case104_seg2.mat', 'Case107/Case107_seg10.mat', 'Case105/Case105_seg3.mat', 'Case105/Case105_seg1.mat', 'Case105/Case105_seg2.mat', 'Case107/Case107_seg7.mat', 'Case107/Case107_seg8.mat', 'Case107/Case107_seg3.mat', 'Case117/Case117_seg2.mat', 'Case107/Case107_seg5.mat', 'Case118/Case118_seg1.mat', 'Case116/Case116_seg6.mat', 'Case116/Case116_seg7.mat', 'Case118/Case118_seg5.mat', 'Case118/Case118_seg6.mat', 'Case118/Case118_seg7.mat', 'Case118/Case118_seg

In [11]:
dtd.count()

57

In [ ]:
print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()

num exec 57


In [ ]:
import datetime
print(datetime.datetime.now())

testdtd=dtd.map(lambda e: mockFunc(e))
#testdtd=dtd.map(lambda e: testFunc(e))
#testdtd.cache()
#%time lres=testdtd.take(len(lsub))
%time lres=testdtd.collect()
#%time lsrc=dtd.collect()
#print(lsrc,len(lsrc))
print(lres,len(lres)) 

print(datetime.datetime.now())

2017-07-21 11:30:19.129404


In [ ]:
import numpy as np
output = np.array(lres)
for line in output[np.argsort(output[:,2])]:
    print(line[2],line[3],line[8])

In [ ]:
1 == 2

In [ ]:
# Test Func
runt=False
if (runt):
    import datetime
    print(datetime.datetime.now())

    #testdtd=dtd.map(lambda e: mockFunc(e))
    testdtd=dtd.map(lambda e: testFunc(e))
    #testdtd.cache()
    #%time lres=testdtd.take(len(lsub))
    %time lres=testdtd.collect()
    #%time lsrc=dtd.collect()
    #print(lsrc,len(lsrc))
    print(lres,len(lres)) 

    print(datetime.datetime.now())

In [ ]:
    print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
    n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()

In [ ]:
import datetime
print(datetime.datetime.now())

testdtd=dtd.map(lambda e: mockFunc(e))
#testdtd=dtd.map(lambda e: testFunc(e))
#testdtd.cache()
#%time lres=testdtd.take(len(lsub))
%time lres=testdtd.collect()
#%time lsrc=dtd.collect()
#print(lsrc,len(lsrc))
print(lres,len(lres)) 

print(datetime.datetime.now())

In [ ]:
import numpy as np
output = np.array(lres)
for line in output[np.argsort(output[:,2])]:
    print(line[2],line[3],line[8])

In [ ]:
!echo $HOME

In [ ]:
!ls $SPARK_CONFIG_HOME

In [ ]:
import os  
os.listdir(os.environ['SPARK_CONFIG_HOME'])

In [ ]:
def readSparkConfig(spark):  
    """
    A function to read the Spark Config of DSX
    :param spark: a string specifying which `spark-defaults.conf` to read. 
    :return: a dict, where the keys corespond to the parameters of `spark-defaults.conf` and the values are the values.
    """
    if not spark in os.listdir(os.environ['SPARK_CONFIG_HOME']):
        #print "'%s' is not a valid option- valid options are: " % (spark, ", ".join(os.listdir(os.environ['SPARK_CONFIG_HOME'])))
        return None
    with open(os.environ['SPARK_CONFIG_HOME'] + "/%s/spark-defaults.conf" % spark, 'r') as f:
        spark_conf = f.read()
    spark_conf_dict = {line.split( )[0] : line.split( )[1] for line in spark_conf.split("\n") if len(line.split( )) > 0}
    return spark_conf_dict

In [ ]:
test = readSparkConfig('spark21master-python3') 

In [ ]:
test['spark.files'] = '/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/full_pipeline.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/get_features.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/fcn_shannon_entro.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/mtspec.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/runtime.py'

In [ ]:
!pwd

In [ ]:
test

In [ ]:
def writeSparkConfig(spark, spark_conf_dict):  
    """
    A function to write the updated Spark Config of DSX (overwrites previous config)
    :param spark: a string specifying which `spark-defaults.conf` to write. 
    :return: None 
    """

    if not spark in os.listdir(os.environ['SPARK_CONFIG_HOME']):
        #print "'%s' is not a valid option- valid options are: " % (spark, ", ".join(os.listdir(os.environ['SPARK_CONFIG_HOME'])))
        return None
    with open(os.environ['SPARK_CONFIG_HOME'] + "/%s/spark-defaults.conf" % spark, 'w') as f:
        for k,v in spark_conf_dict.items():
            print(k + "\t" + v + "\n")
            ln=k + "\t" + v + "\n"
            f.write(ln)
    #print "Successfully wrote new configuration to %s" % (os.environ['SPARK_CONFIG_HOME'] + "/%s/spark-defaults.conf" % spark)
    return None

In [ ]:
writeSparkConfig('spark21master-python3', test)

In [ ]:
!cp -rf $SPARK_CONFIG_HOME/spark21master-python3 $SPARK_CONFIG_HOME/spark21master-python3_bu; ls $SPARK_CONFIG_HOME

In [ ]:
!printenv